In [ ]:
import sys
import math
import warnings
warnings.filterwarnings("ignore", message=".*The 'nopython' keyword.*")
import shap
import joblib
# from sklearn.externals import joblib
import mlflow
import random
import pickle
import pandas_profiling as pp
# import mlflow.sklearn
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error
from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
import lightgbm as lgb 
import xgboost as xgb
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor , VotingRegressor
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler, LabelEncoder, QuantileTransformer
from sklearn.feature_extraction import FeatureHasher
from category_encoders import BinaryEncoder 
from sklearn.impute import SimpleImputer
%matplotlib inline

In [2]:
df_train = pd.read_csv("D:\work\Hackathon\CO2_Estimation\CO2_Hackathon_Trial\data_folder\co2_emmision_train_data.csv")
df_test = pd.read_csv("D:\work\Hackathon\CO2_Estimation\CO2_Hackathon_Trial\data_folder\co2_emmision_test_data.csv")

In [3]:
def printCategorical_Numerical_Column(df):
    category_columns=df.select_dtypes(include=['object']).columns
    numerical_columns = df.select_dtypes(include=np.number).columns.tolist()
    print("Categorical Variables:")
    print(category_columns)
    print("Numerical Variables:")
    print(numerical_columns)
    return category_columns, numerical_columns

In [4]:
def getUniqueClassesCategory(df, category_column):
    for feature in category_column:
        print(feature)
        print("Number of unique features : ", len(df[feature].unique()))

In [5]:
def  getUniqueClassesCategoryDetails(df,category_column):
    for feature in category_column:
        print(feature)
        print("Number of unique features : ", df[feature].unique())
        print("*"*100)

In [6]:
def combineFuelType(df):
    return f'{df["Fuel Type"]}_{df["Fuel Type 1"]}_{df["Fuel Type 2"]}'    

In [7]:
def combineFuelConsumption(df):
    return df['Fuel Consumption Combined Le/100 km'].str.split(' ', n=1).str[0]

In [8]:
def dropColumns(df):
    df.drop("Fuel Type 1", axis='columns', inplace=True)
    df.drop("Fuel Type 2", axis='columns',inplace=True)
    df.drop("Fuel Type", axis='columns',inplace=True)
    df.drop("Fuel Consumption [City (L/100 km)]", axis='columns',inplace=True)
    df.drop("Fuel Consumption [Hwy (L/100 km)]", axis='columns',inplace=True)
    df.drop("Consumption [City (Le/100 km)]", axis='columns',inplace=True)
    df.drop("Consumption [Hwy (Le/100 km)]", axis='columns',inplace=True)
    df.drop("Range 1 (km)", axis='columns',inplace=True)
    df.drop("Range 2 (km)", axis='columns',inplace=True)
    df.drop("Year",axis='columns',inplace=True)
    df.drop("Model",axis='columns',inplace=True)
    df.drop("Fuel Consumption [City (kWh/100 km)]", axis='columns',inplace=True)
    return df

In [9]:
def displayDFInfo(df):
    df.info()

In [10]:
def removeSpecialCharacters(df, categorical_cols):
    for column in categorical_cols:
        df[column] = ((df[column].str.replace('[^a-zA-Z0-9\s]','',regex=True)).map(str. lower)).str.replace(" ","")
    if column == 'Combined Fuel Type' :
        df[column] = df[column].str.replace('[nan]', '', regex=True)
    return df

In [11]:
def cleaningMakeVehicleClass(df):
    Incorrect_2_Correct = {'compatc':'compact','fullsiez':'fullsize','midsiez':'midsize','minicompatc':'minicompact','minivna':'minivan','pickuptruckstandadr':'pickuptruckstandard', 'specialpurposevehicel':'specialpurposevehicle', 'stationwagonmidsiez':'stationwagonmidsize', 'subcompatc':'subcompact','svu':'suv','suvstandadr':'suvstandard','vancarog':'vancargo','vanpassengre':'vanpassenger','twoseatre':'twoseater'}
    Incorrect_2_Correct_Make = {'acuar':'acura','alfaromoe':'alfaromeo','astonmartni':'astonmartin','auid':'audi','bentlye':'bentley','bwm':'bmw','buikc':'buick','cadillca':'cadillac','chevrolte':'chevrolet','chryslre':'chrysler','dodeg':'dodge','eagel':'eagle','ferrair':'ferrari','fita':'fiat','fodr':'ford','gcm':'gmc','genessi':'genesis','honad':'honda','hyundia':'hyundai','infiniit':'infiniti','jagura':'jaguar','jepe':'jeep','kai':'kia','lamborghiin':'lamborghini','landrovre':'landrover','lexsu':'lexus','linconl':'lincoln','maserait':'maserati','mazad':'mazda','mercedesbezn':'mercedesbenz','mercuyr':'mercury','miin':'mini','mitsubisih':'mitsubishi','nissna':'nissan','oldsmobiel':'oldsmobile','plymouht':'plymouth','pontica':'pontiac','porsceh':'porsche','rma':'ram','rollsroyec':'rollsroyce','saab':'saba','satunr':'saturn','scino':'scion','subaur':'subaru','suzuik':'suzuki','toyoat':'toyota','volkswagne':'volkswagen','volov':'volvo','tesal':'tesla','smatr':'smart'}
    for key, value in Incorrect_2_Correct_Make.items():
        df['Make'] = df['Make'].str.replace(key, value)

    for key, value in Incorrect_2_Correct.items():
        df['Vehicle Class'] = df['Vehicle Class'].str.replace(key, value)

    return df


In [12]:
def convertObject2Float(df, column):
    df[column] =  pd.to_numeric(df[column], errors='coerce')
    return df

In [13]:
def applyDataCleaningTotal(df, set_name = 'Train'):
    
    category_column_pre, numerical_column_pre = printCategorical_Numerical_Column(df)

    print(category_column_pre)
    print(numerical_column_pre)

    getUniqueClassesCategory(df, category_column_pre)
    getUniqueClassesCategoryDetails(df,category_column_pre)

    df['Combined Fuel Type'] = df.apply(combineFuelType, axis=1)
    df['Fuel Consumption Combined Le/100 km'] = combineFuelConsumption(df)
    
    # df = Transmission2category(df)

    df = dropColumns(df)
    df = convertObject2Float(df, 'Fuel Consumption Combined Le/100 km')
    
    print("After Cleaning")
    print("*/*"*50)
    category_column_post, numerical_column_post = printCategorical_Numerical_Column(df)
    print('--'*50)
    print(category_column_post)
    print(numerical_column_post)
    print('--'*50)
    df = removeSpecialCharacters(df, category_column_post)
    df = cleaningMakeVehicleClass(df)
    getUniqueClassesCategoryDetails(df,category_column_post)

    ## Fill missing values with zero 
    df.fillna(0, inplace=True)
    if set_name == 'Train':
        df = df.drop_duplicates()
        
    getUniqueClassesCategory(df, category_column_post)
    return df, category_column_post, numerical_column_post, category_column_pre, numerical_column_pre
    
    

In [14]:
df_train_modified, category_column_train_post, numerical_column_train_post, category_column_train_pre, numerical_column_train_pre = applyDataCleaningTotal(df_train)
df_test_modified, category_column_test_post, numerical_column_test_post, category_column_test_pre, numerical_column_test_pre = applyDataCleaningTotal(df_test, set_name='Test')

Categorical Variables:
Index(['Make', 'Model', 'Vehicle Class', 'Transmission', 'Fuel Type',
       'Fuel Type 1', 'Fuel Consumption Combined Le/100 km', 'Fuel Type 2'],
      dtype='object')
Numerical Variables:
['Year', 'Engine Size (L)', 'Cylinders', 'Fuel Consumption [City (L/100 km)]', 'Fuel Consumption [Hwy (L/100 km)]', 'Fuel Consumption [Comb (L/100 km)]', 'Fuel Consumption [Comb (mpg)]', 'CO2 Emissions (g/km)', 'Motor (kW)', 'Range 1 (km)', 'Recharge Time (h)', 'Range 2 (km)', 'Fuel Consumption [City (kWh/100 km)]', 'Fuel Consumption [Comb (kWh/100 km)]', 'Consumption [City (Le/100 km)]', 'Consumption [Hwy (Le/100 km)]', 'Consumption [Comb (Le/100 km)]', 'Range (km)']
Index(['Make', 'Model', 'Vehicle Class', 'Transmission', 'Fuel Type',
       'Fuel Type 1', 'Fuel Consumption Combined Le/100 km', 'Fuel Type 2'],
      dtype='object')
['Year', 'Engine Size (L)', 'Cylinders', 'Fuel Consumption [City (L/100 km)]', 'Fuel Consumption [Hwy (L/100 km)]', 'Fuel Consumption [Comb (L/10

In [15]:
df_train_modified['Transmission'].unique()

array(['am8', 'as10', 'am9', 'a8', 'a9', 'am7', 'as8', 'm6', 'as6', 'av',
       'as9', 'a10', 'a6', 'm5', 'm7', 'av7', 'av1', 'am6', 'as7', 'av8',
       'av6', 'av10', 'as5', 'a7', 'a5', 'a4', 'as4', 'am5', 'm4', 'a3',
       'a1', 'a2'], dtype=object)

In [16]:
# profile_train = pp.ProfileReport(df_train_modified, title="CO2 Emmision EDA Report Train",n_freq_table_max=250)
# # profile
# # save the profile report to an HTML file
# profile_train.to_file("CO2_Emmision_eda_report_Train_Data.html")

# profile_test = pp.ProfileReport(df_test_modified, title="CO2 Emmision EDA Report Test",n_freq_table_max=250)
# # profile
# # save the profile report to an HTML file
# profile_test.to_file("CO2_Emmision_eda_report_Test_Data.html")

In [17]:
# Split the Training Data in to Train and Validation
X = df_train_modified.loc[:,df_train_modified.columns != 'CO2 Emissions (g/km)']
y = df_train_modified.loc[:,df_train_modified.columns == 'CO2 Emissions (g/km)']

In [18]:
category_columns_train_post_split,numerical_columns_train_post_split =  printCategorical_Numerical_Column(X)

Categorical Variables:
Index(['Make', 'Vehicle Class', 'Transmission', 'Combined Fuel Type'], dtype='object')
Numerical Variables:
['Engine Size (L)', 'Cylinders', 'Fuel Consumption [Comb (L/100 km)]', 'Fuel Consumption [Comb (mpg)]', 'Motor (kW)', 'Fuel Consumption Combined Le/100 km', 'Recharge Time (h)', 'Fuel Consumption [Comb (kWh/100 km)]', 'Consumption [Comb (Le/100 km)]', 'Range (km)']


In [19]:
x_test = df_test_modified

In [20]:
x_train, x_val, y_train, y_val = train_test_split(X, y.values.ravel(), test_size=0.2, random_state=42, shuffle=True)

In [21]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12111 entries, 7870 to 12111
Data columns (total 14 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Make                                  12111 non-null  object 
 1   Vehicle Class                         12111 non-null  object 
 2   Engine Size (L)                       12111 non-null  float64
 3   Cylinders                             12111 non-null  float64
 4   Transmission                          12111 non-null  object 
 5   Fuel Consumption [Comb (L/100 km)]    12111 non-null  float64
 6   Fuel Consumption [Comb (mpg)]         12111 non-null  float64
 7   Motor (kW)                            12111 non-null  float64
 8   Fuel Consumption Combined Le/100 km   12111 non-null  float64
 9   Recharge Time (h)                     12111 non-null  float64
 10  Fuel Consumption [Comb (kWh/100 km)]  12111 non-null  float64
 11  Consumption 

In [22]:
# Define transformers for numerical and categorical data
numerical_transformer = Pipeline(steps=[
    ('RobustScaler', RobustScaler())  # Scale features to have mean=0 and std=1
])

categorical_transformer = Pipeline(steps=[
    ('BinaryEncoder', BinaryEncoder())  # One-hot encode categorical features
])

In [23]:
# Combine transformers using ColumnTransformer
preprocessor_pipeline = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_columns_train_post_split),
        ('cat', categorical_transformer, category_columns_train_post_split)
    ])

In [24]:
models = [
    # ('Dummy Regression', DummyRegressor(),{}),

    # ('Linear Regression', LinearRegression(),{
    #     'model__fit_intercept': [True, False]
    # }),
    # ('Ridge Regression', Ridge(),{
    #     'model__alpha': [0.01, 0.1, 1.0],
    # }),

    # ('Lasso Regression', Lasso(alpha=1.0),{
    #      'model__alpha': [0.01, 0.1, 1.0],
    # }),

    # ('Random Forest', RandomForestRegressor(random_state=42), {
    #     'model__n_estimators': [25,50,75,100, 200],
    #     'model__max_depth': [None, 10, 20, 30],
    #     'model__min_samples_split': [2, 5, 10],
    #     'model__min_samples_leaf': [1, 2, 4],
    #     'model__max_features': [None, 'auto', 'sqrt', 'log2'],
    # }),

    # ('Gradient Boosting', GradientBoostingRegressor(random_state=42, verbose=2), {
    #     'model__loss': ['absolute_error', 'huber', 'quantile','squared_error'],
    #     'model__n_estimators': [300, 400, 500, 600],
    #     'model__learning_rate': [0.1, 0.2, 0.4],
    #     'model__max_depth': [4, 5],
    #     'model__min_samples_split': [2, 5, 10, 20],
    #     'model__max_features': ['auto', 'sqrt', 'log2'],
    # }),

    # ('Light GBM', lgb.LGBMRegressor(objective='regression',random_state=42, verbose=2), {
    #     'model__learning_rate': [0.4 ,0.3, 0.2, 0.1],
    #     'model__n_estimators': [100, 150, 200, 300],
    #     'model__max_depth': [4, 6, 8],
    #     'model__colsample_bytree': [0.7, 0.8, 0.9],
    #     'model__subsample': [0.7, 0.8, 0.9],
    #     'model__min_child_samples': [1, 5, 10]
    # })

    # ('XGB Regressor', xgb.XGBRFRegressor(random_state=42),{
    #     'model__objective':['reg:linear'],
    #     'model__learning_rate': [0.01, 0.1, 0.2],
    #     'model__max_depth': [5, 6, 7],
    #     'model__min_child_weight': [4],
    #     'model__silent': [1],
    #     'model__subsample': [0.8, 0.9, 1.0],
    #     'model__colsample_bytree': [0.7],
    #     'model__n_estimators': [100, 200, 300]
    #  }),
    # ('KNN Regressor', KNeighborsRegressor(),{
    #     'model__n_neighbors': [3, 5, 7, 9],
    #     'model__weights': ['uniform', 'distance'],
    #     'model__p': [1, 2]
    # }),
    # ('SVR Regressor', SVR(), {
    #     'model__C': [0.1, 1, 10],
    #     'model__kernel': ['linear', 'rbf', 'poly'],
    #     'model__epsilon': [0.01, 0.1, 1]
    # })

    # ('CatBoost Regressor', CatBoostRegressor(),{
    #     'model__learning_rate': [0.01, 0.1, 0.2],
    #     'model__n_estimators': [100, 200, 300],
    #     'model__depth': [3, 4, 5],
    #     'model__l2_leaf_reg': [1, 3, 5],
    #     'model__subsample': [0.8, 0.9, 1.0],
    # }),
    # ('Decision Tree Regressor',DecisionTreeRegressor(),{
    #     'model__max_depth': [None, 10, 20,30,40,50],
    #     'model__min_samples_split': [2, 5, 10],
    #     'model__min_samples_leaf': [1, 2, 4],
    #     'model__max_features': ['auto', 'sqrt', 'log2'],
    # })
    ('MLP Regressor', MLPRegressor(max_iter=20000, random_state=42, verbose=True),
     {'model__hidden_layer_sizes': [(100, 50, 25)],
      'model__activation': ['relu'],
      'model__solver': ['lbfgs'],
      'model__alpha': [0.01],
      'model__learning_rate': ['adaptive']
     })
    
]

In [25]:
log_file = "MLPRegressor_Model_Pipeline_Experiment_BinaryEncoder_RobustScaler_No_Stratify_neg_mean_squared_error_GridSearch_Result.txt"
# Create a custom logger function to write to both console and file
def custom_logger(message):
    # Print to console
    print(message)
    
    # Log to a file
    with open(log_file, "a") as file:
        file.write(message + "\n")

# Redirect stdout to the log file
sys.stdout = open(log_file, "a")

In [ ]:


results = []
mlflow.set_experiment("MLPRegressor_Model_Pipeline_Experiment_BinaryEncoder_RobustScaler_No_Stratify_neg_mean_squared_error_GridSearch")
Model_Name = []
RMSE = []
R2 = []
ctr = 1
for model_name, model, param_grid in models:

    # x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random.randint(0, 50), stratify=X['Make'])
    # Create a pipeline with preprocessing and the current model
    pipeline = Pipeline(steps=[ ('preprocessor', preprocessor_pipeline),
                                ('model', model)])
    print(model)

    with mlflow.start_run():
        mlflow.log_params({'Model Name':model_name})

        # stratified_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
        # Fit the pipeline to the training data
        grid_search.fit(x_train, y_train)

        best_model = grid_search.best_estimator_
        best_params = grid_search.best_params_

        # Make predictions on the test data
        y_pred = best_model.predict(x_val)

        # Evaluate the model
        mse = mean_squared_error(y_val, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_val, y_pred)

        # CSV file write for test data
        prdicted_filename = f'predicted_results_{model_name}.csv'
        y_pred_test = best_model.predict(x_test)
        print(y_pred_test)
        y_pred_test = np.where(y_pred_test.ravel() < 10, 0, y_pred_test.ravel())
        output=pd.DataFrame({"Predicted_Emmission":y_pred_test.ravel()})
        output.to_csv(prdicted_filename,index=False)
        mlflow.log_artifact(prdicted_filename) 

        # MLFLOW logging the parameters
        mlflow.log_params(grid_search.best_params_)
        print(grid_search.cv_results_)
        mlflow.log_params({'n_splits': grid_search.n_splits_})
        mlflow.log_params({'best_score': grid_search.best_score_})
        
        mlflow.log_metric('RMSE', rmse)
        mlflow.log_metric('R2', r2)

        Model_Name.append(str(model))
        RMSE.append(rmse)
        R2.append(r2)

        model_filename = f'best_model_{model_name}.joblib'
        with open(model_filename, 'wb') as file:
            joblib.dump(best_model, file)

        grid_filename = f'best_model_cv_results_{model_name}.json'
        cv_results_df = pd.DataFrame(grid_search.cv_results_)
        cv_results_json = cv_results_df.to_json(orient='records')
        with open(grid_filename, 'w') as json_file:
            json_file.write(cv_results_json)

        mlflow.log_artifact(grid_filename)
        
        plt.scatter(np.arange(0,len(y_val)),y_val,color='k', label='GT')
        plt.scatter(np.arange(0,len(y_pred)),y_pred,color='g', label='PRED')
        plt.xlabel("Actual Values")
        plt.ylabel("Predicted Values")
        plt.title(f"{model_name}\nR-squared: {r2:.2f}")
        plt.legend()
        plt.savefig(f"{model_name}.png")
        mlflow.log_artifact(f"{model_name}.png")
        plt.close()

        custom_logger(f"Model: {model_name}")
        custom_logger(f"Best Hyperparameters: {best_params}")
        custom_logger(f"Mean Squared Error: {mse:.2f}")
        custom_logger(f"Root Mean Squared Error: {rmse:.2f}")
        custom_logger(f"R-squared: {r2:.2f}")
        custom_logger('-' * 200)
        
        print(f"Model: {model_name}")
        print(f"Best Hyperparameters: {best_params}")
        print(f"Mean Squared Error: {mse:.2f}")
        print(f"Root Mean Squared Error: {rmse:.2f}")
        print(f"R-squared: {r2:.2f}")
        print('-' * 200)

        mlflow.sklearn.log_model(grid_search.best_estimator_, "best_model")
        ctr = ctr+1
        


sys.stdout.close()
sys.stdout = sys.__stdout__
plt.tight_layout()
mlflow.log_artifact(log_file)
plt.show()
mlflow.end_run()